# 04 - DBSCAN

## Introdução

O **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise) é um algoritmo de clustering baseado em densidade desenvolvido por Martin Ester e colaboradores em 1996. Diferentemente do K-Means e clustering hierárquico, o DBSCAN não requer que especifiquemos o número de clusters antecipadamente e é capaz de identificar clusters de formas arbitrárias e detectar outliers (ruído).

### Características Principais do DBSCAN:

1. **Baseado em densidade**: Agrupa pontos que estão densamente empacotados
2. **Detecção de ruído**: Identifica automaticamente outliers
3. **Forma arbitrária**: Pode encontrar clusters de qualquer formato
4. **Número automático de clusters**: Não precisa especificar k antecipadamente
5. **Robusto a outliers**: Outliers não afetam a formação dos clusters

## Fundamentos Matemáticos

O DBSCAN utiliza dois parâmetros principais:
- $\varepsilon$ (eps): raio da vizinhança
- $\text{minPts}$: número mínimo de pontos para formar um cluster

### Definições Fundamentais:

**1. Vizinhança-$\varepsilon$**: Para um ponto $p$, sua vizinhança-$\varepsilon$ é definida como:
$$N_\varepsilon(p) = \{q \in D | \text{dist}(p,q) \leq \varepsilon\}$$

**2. Ponto Central (Core Point)**: Um ponto $p$ é um ponto central se:
$$|N_\varepsilon(p)| \geq \text{minPts}$$

**3. Diretamente Alcançável por Densidade**: Um ponto $q$ é diretamente alcançável por densidade a partir de $p$ se:
- $q \in N_\varepsilon(p)$ e
- $p$ é um ponto central

**4. Alcançável por Densidade**: Um ponto $q$ é alcançável por densidade a partir de $p$ se existe uma cadeia de pontos $p_1, p_2, ..., p_n$ onde $p_1 = p$ e $p_n = q$, tal que $p_{i+1}$ é diretamente alcançável por densidade a partir de $p_i$.

**5. Conectado por Densidade**: Dois pontos $p$ e $q$ são conectados por densidade se existe um ponto $o$ tal que tanto $p$ quanto $q$ são alcançáveis por densidade a partir de $o$.

### Classificação dos Pontos:

- **Core Point (Ponto Central)**: $|N_\varepsilon(p)| \geq \text{minPts}$
- **Border Point (Ponto de Fronteira)**: $|N_\varepsilon(p)| < \text{minPts}$, mas está na vizinhança de um core point
- **Noise Point (Ponto de Ruído)**: Não é core nem border point

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN as SklearnDBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
import pandas as pd

plt.style.use('seaborn-v0_8-whitegrid')
np.random.seed(1)

## Implementação do DBSCAN

Vamos implementar o algoritmo DBSCAN passo a passo usando apenas NumPy:

In [ ]:
class DBSCAN:
    def __init__(self, eps=0.5, min_pts=5, metric='euclidean'):
        """Inicializa o DBSCAN com os parâmetros eps e min_pts"""
        self.eps = eps
        self.min_pts = min_pts
        self.metric = metric
        self.labels_ = None
        self.core_samples_ = None
        self.n_clusters_ = 0

    def _calculate_distance_matrix(self, X):
        """Calcula a matriz de distâncias entre todos os pontos"""
        if self.metric == 'euclidean':
            distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)
        # elif self.metric == '...':
            # distance = ...
        else:
            raise ValueError("Métrica não suportada")
        return distances

    def _get_neighbors(self, point_idx, distance_matrix):
        """Encontra todos os vizinhos dentro da distância eps"""
        return np.where(distance_matrix[point_idx] <= self.eps)[0]

    def _expand_cluster(self, point_idx, neighbors, cluster_id, distance_matrix, visited, labels):
        """Expande o cluster a partir do ponto inicial"""
        labels[point_idx] = cluster_id
        queue = neighbors.tolist()

        while queue:
            neighbor_idx = queue.pop()

            if not visited[neighbor_idx]:
                visited[neighbor_idx] = True
                neighbor_neighbors = self._get_neighbors(neighbor_idx, distance_matrix)

                if len(neighbor_neighbors) >= self.min_pts:
                    queue.extend(neighbor_neighbors)

            if labels[neighbor_idx] == -1:
                labels[neighbor_idx] = cluster_id

    def fit(self, X):
        """Executa o algoritmo DBSCAN"""
        n_points = len(X)
        visited = np.zeros(n_points, dtype=bool)
        cluster_id = 0
        self.labels_ = np.full(n_points, -1)  # -1 = ruído
        self.core_samples_ = []

        distance_matrix = self._calculate_distance_matrix(X)

        for point_idx in range(n_points):
            if visited[point_idx]:
                continue

            visited[point_idx] = True
            neighbors = self._get_neighbors(point_idx, distance_matrix)

            if len(neighbors) >= self.min_pts:   # core point
                self.core_samples_.append(point_idx)
                self._expand_cluster(point_idx, neighbors, cluster_id, distance_matrix, visited, self.labels_)
                cluster_id += 1

        self.core_samples_ = np.array(self.core_samples_)
        self.n_clusters_ = cluster_id
        return self

    def fit_predict(self, X):
        """Executa DBSCAN e retorna os labels"""
        self.fit(X)
        return self.labels_

## Demonstração com Dados Sintéticos

Vamos criar dados sintéticos para demonstrar o funcionamento do DBSCAN:

In [ ]:
rng = np.random.default_rng(42)

# Cabeça: círculo com leve ruído
n_head = 400
theta = rng.uniform(0, 2*np.pi, n_head)
R = 10 + rng.normal(0, 0.35, n_head)
head = np.c_[R*np.cos(theta), R*np.sin(theta)]
y_head = np.full(n_head, 0)

# Olhos: dois blobs gaussianos
n_eye = 100
eye_left  = rng.normal(loc=[-3.2,  3.0], scale=[0.45, 0.45], size=(n_eye//2, 2))
eye_right = rng.normal(loc=[ 3.2,  3.0], scale=[0.45, 0.45], size=(n_eye - n_eye//2, 2))
eyes = np.vstack([eye_left, eye_right])
y_eyes = np.full(eyes.shape[0], 1)

# Boca: arco inferior com jitter (sorriso)
n_mouth = 100
phi = rng.uniform(np.deg2rad(200), np.deg2rad(340), n_mouth)  # arco de 200° a 340°
Rm = 5 + rng.normal(0, 0.22, n_mouth)
mouth = np.c_[Rm*np.cos(phi), -1 + Rm*np.sin(phi)]
mouth += rng.normal(0, [0.12, 0.15], mouth.shape)  # engrossar um pouco
y_mouth = np.full(n_mouth, 2)

# Ruído: pontos aleatórios
n_noise = 100
noise = rng.uniform(low=[-13, -13], high=[13, 13], size=(n_noise, 2))
y_noise = np.full(n_noise, -1)

# Concatenar
X_synthetic = np.vstack([head, eyes, mouth, noise])
true_labels  = np.concatenate([y_head, y_eyes, y_mouth, y_noise])

In [ ]:
# Visualizar os dados sintéticos
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
colors = ['red', 'blue', 'green', 'gray']
for i in range(4):
    if i == 3:  # ruído
        mask = true_labels == -1
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=colors[i], alpha=0.6, s=30, marker='x', label='Ruído')
    else:
        mask = true_labels == i
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=colors[i], alpha=0.7, s=50, label=f'Cluster {i+1}')

plt.title('Dados Sintéticos - Classes Verdadeiras')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(X_synthetic[:, 0], X_synthetic[:, 1], c='black', alpha=0.6, s=30)
plt.title('Dados Sintéticos - Sem Labels')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
dbscan = DBSCAN(eps=1.3, min_pts=7)
labels = dbscan.fit_predict(X_synthetic)

plt.figure(figsize=(6, 6))
unique_labels = np.unique(labels)
colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))

for k, label in enumerate(unique_labels):
    if label == -1:
        # Ruído
        mask = labels == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c='black', marker='x', s=30, alpha=0.6, label='Ruído')
    else:
        # Clusters
        mask = labels == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1], c=[colors[k]], s=50, alpha=0.7, label=f'Cluster {label}')

# Destacar core points
if len(dbscan.core_samples_) > 0:
    plt.scatter(X_synthetic[dbscan.core_samples_, 0],
                X_synthetic[dbscan.core_samples_, 1],
                s=100, facecolors='none', edgecolors='black',
                linewidth=2, alpha=0.8)

plt.tight_layout()
plt.show()

## Aplicando DBSCAN aos Dados Sintéticos

Agora vamos aplicar nosso algoritmo DBSCAN:

In [ ]:
# Testar diferentes valores de eps e min_pts
eps_values = [0.5, 1.3, 2.0]
min_pts_values = [3, 5, 8]

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('DBSCAN com Diferentes Parâmetros', fontsize=16)

for i, eps in enumerate(eps_values):
    for j, min_pts in enumerate(min_pts_values):
        # Aplicar DBSCAN
        dbscan = DBSCAN(eps=eps, min_pts=min_pts)
        labels = dbscan.fit_predict(X_synthetic)

        # Visualizar resultados
        ax = axes[i, j]

        unique_labels = np.unique(labels)
        colors = plt.cm.Set1(np.linspace(0, 1, len(unique_labels)))

        for k, label in enumerate(unique_labels):
            if label == -1:
                # Ruído
                mask = labels == label
                ax.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                          c='black', marker='x', s=30, alpha=0.6, label='Ruído')
            else:
                # Clusters
                mask = labels == label
                ax.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                          c=[colors[k]], s=50, alpha=0.7, label=f'Cluster {label}')

        # Destacar core points
        if len(dbscan.core_samples_) > 0:
            ax.scatter(X_synthetic[dbscan.core_samples_, 0],
                      X_synthetic[dbscan.core_samples_, 1],
                      s=100, facecolors='none', edgecolors='black',
                      linewidth=2, alpha=0.8)

        ax.set_title(f'eps={eps}, min_pts={min_pts}\n{dbscan.n_clusters_} clusters')
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Análise com Parâmetros Ótimos

Vamos escolher os parâmetros que melhor capturam a estrutura dos dados:

In [ ]:
# Parâmetros que parecem funcionar melhor
best_eps = 1.3
best_min_pts = 5

# Aplicar DBSCAN com os melhores parâmetros
dbscan_best = DBSCAN(eps=best_eps, min_pts=best_min_pts)
labels_best = dbscan_best.fit_predict(X_synthetic)

# Análise detalhada
unique_labels = np.unique(labels_best)
n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
n_noise = np.sum(labels_best == -1)
n_core_samples = len(dbscan_best.core_samples_)

print(f"Resultados do DBSCAN (eps={best_eps}, min_pts={best_min_pts}):")
print(f"- Número de clusters encontrados: {n_clusters}")
print(f"- Número de pontos de ruído: {n_noise}")
print(f"- Número de core samples: {n_core_samples}")
print(f"- Labels únicos: {unique_labels}")

In [ ]:
# Classificar pontos por tipo
core_mask = np.zeros(len(X_synthetic), dtype=bool)
if len(dbscan_best.core_samples_) > 0:
    core_mask[dbscan_best.core_samples_] = True

border_mask = (labels_best != -1) & (~core_mask)
noise_mask = labels_best == -1

print(f"Classificação dos pontos:")
print(f"- Core points: {np.sum(core_mask)}")
print(f"- Border points: {np.sum(border_mask)}")
print(f"- Noise points: {np.sum(noise_mask)}")

In [ ]:
# Visualização detalhada dos tipos de pontos
plt.figure(figsize=(15, 5))

# Subplot 1: Clusters encontrados
plt.subplot(1, 3, 1)
colors = plt.cm.Set1(np.linspace(0, 1, max(len(unique_labels), 3)))

for i, label in enumerate(unique_labels):
    if label == -1:
        mask = labels_best == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                   c='black', marker='x', s=50, alpha=0.8, label='Ruído')
    else:
        mask = labels_best == label
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                   c=[colors[i]], s=60, alpha=0.8, label=f'Cluster {label}')

plt.title(f'DBSCAN - Clusters Encontrados\n{n_clusters} clusters, {n_noise} ruído')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Tipos de pontos
plt.subplot(1, 3, 2)
plt.scatter(X_synthetic[core_mask, 0], X_synthetic[core_mask, 1],
           c='red', s=80, alpha=0.8, label='Core Points', marker='o')
plt.scatter(X_synthetic[border_mask, 0], X_synthetic[border_mask, 1],
           c='blue', s=60, alpha=0.8, label='Border Points', marker='s')
plt.scatter(X_synthetic[noise_mask, 0], X_synthetic[noise_mask, 1],
           c='black', s=40, alpha=0.8, label='Noise Points', marker='x')

plt.title('Classificação dos Pontos')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Comparação com ground truth
plt.subplot(1, 3, 3)
for i in range(4):
    if i == 3:  # ruído
        mask = true_labels == -1
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                   c='gray', alpha=0.6, s=30, marker='x', label='Ruído Real')
    else:
        mask = true_labels == i
        plt.scatter(X_synthetic[mask, 0], X_synthetic[mask, 1],
                   c=colors[i], alpha=0.7, s=50, label=f'Cluster Real {i+1}')

plt.title('Ground Truth')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Estimativa do Parâmetro $\varepsilon$ usando K-Distance

Uma das maiores dificuldades do DBSCAN é escolher o valor apropriado para o parâmetro $\varepsilon$ (eps). O método **K-Distance** é uma heurística eficaz para estimar este parâmetro.

O método K-Distance consiste em:

1. **Calcular a k-ésima distância mais próxima** para cada ponto no dataset
2. **Ordenar essas distâncias** em ordem decrescente
3. **Identificar o "cotovelo"** no gráfico resultante

A intuição é que pontos dentro de clusters densos terão k-ésimas distâncias pequenas, enquanto pontos de ruído ou em bordas de clusters terão distâncias maiores.

### Algoritmo K-Distance:

Para um dataset $D$ e parâmetro $k = \text{minPts} - 1$:

1. Para cada ponto $p_i \in D$:
   - Calcule $d_k(p_i)$ = distância ao k-ésimo vizinho mais próximo
2. Ordene os valores $d_k(p_i)$
3. Plote o gráfico K-Distance
4. Escolha $\varepsilon$ no ponto onde a curva tem maior curvatura (cotovelo)

In [ ]:
from sklearn.neighbors import NearestNeighbors

def plot_k_distance(X, min_pts, title="K-Distance Plot"):
    """Plota o gráfico K-Distance usando sklearn.NearestNeighbors."""
    k = int(min_pts - 1)

    nn = NearestNeighbors(n_neighbors=k+1, metric="euclidean")
    nn.fit(X)
    distances, _ = nn.kneighbors(X)

    kth_distances = distances[:, k]
    k_distances_sorted = np.sort(kth_distances)

    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances_sorted)), k_distances_sorted, linewidth=2, label=f'{k}-distance')
    plt.xlabel("Pontos ordenados por distância")
    plt.ylabel(f"{k}-distance")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_k_distance(X_synthetic, min_pts=5, title="K-Distance Plot para Dados Sintéticos")

## Vantagens e Desvantagens do DBSCAN

### Vantagens:

1. **Não requer especificar o número de clusters antecipadamente**
2. **Pode encontrar clusters de forma arbitrária** (não apenas esféricos)
3. **Identifica automaticamente outliers/ruído**
4. **Robusto a outliers** (não afetam a formação dos clusters)
5. **Determinístico** (sempre produz os mesmos resultados)

### Desvantagens:

1. **Sensível aos parâmetros** eps e min_pts
2. **Dificuldade com clusters de densidades diferentes**
3. **Problemas em alta dimensionalidade** ("curse of dimensionality")
4. **Complexidade computacional** O(n²) no pior caso
5. **Requer escolha cuidadosa da métrica de distância**

## Exercícios

### Exercício 1: Ajuste de Parâmetros no DBSCAN em 3D

Com os dados das **três esferas concêntricas**, realize:

1. Plotar o K-Distance para diferentes valores de `min_pts` e sugerir um intervalo adequado para `eps`.
2. Selecionar os melhores parâmetros de `min_pts` e `eps`.
3. Visualizar em 3D os clusters encontrados (cores diferentes) e comentar a escolha de `eps` e `min_samples`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist, squareform
import warnings
warnings.filterwarnings("ignore")

class DBSCAN:
    def __init__(self, eps=0.5, min_pts=5, metric='euclidean'):
        self.eps = eps
        self.min_pts = min_pts
        self.metric = metric
        self.labels_ = None
        self.core_samples_ = None
        self.n_clusters_ = 0

    def _calculate_distance_matrix(self, X):
        if self.metric == 'euclidean':
            distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)
        elif self.metric == 'precomputed':
            return X
        else:
            raise ValueError("Métrica não suportada ou não implementada")
        return distances

    def _get_neighbors(self, point_idx, distance_matrix):
        return np.where(distance_matrix[point_idx] <= self.eps)[0]

    def _expand_cluster(self, point_idx, neighbors, cluster_id, distance_matrix, visited, labels):
        labels[point_idx] = cluster_id
        queue = neighbors.tolist()

        while queue:
            neighbor_idx = queue.pop()

            if not visited[neighbor_idx]:
                visited[neighbor_idx] = True
                neighbor_neighbors = self._get_neighbors(neighbor_idx, distance_matrix)

                if len(neighbor_neighbors) >= self.min_pts:
                    queue.extend(neighbor_neighbors)

            if labels[neighbor_idx] == -1:
                labels[neighbor_idx] = cluster_id

    def fit(self, X):
        n_points = len(X)
        visited = np.zeros(n_points, dtype=bool)
        cluster_id = 0
        self.labels_ = np.full(n_points, -1)
        self.core_samples_ = []

        distance_matrix = self._calculate_distance_matrix(X)

        for point_idx in range(n_points):
            if visited[point_idx]:
                continue

            visited[point_idx] = True
            neighbors = self._get_neighbors(point_idx, distance_matrix)

            if len(neighbors) >= self.min_pts:
                self.core_samples_.append(point_idx)
                self._expand_cluster(point_idx, neighbors, cluster_id, distance_matrix, visited, self.labels_)
                cluster_id += 1

        self.core_samples_ = np.array(self.core_samples_)
        self.n_clusters_ = cluster_id
        return self

    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

def generate_concentric_spheres(radii=[3, 15], n_samples_per_sphere=1000, noise=0.2, random_state=42):
    rng = np.random.default_rng(random_state)
    X, y = [], []

    for i, r in enumerate(radii):
        phi = rng.uniform(0, 2*np.pi, n_samples_per_sphere)
        costheta = rng.uniform(-1, 1, n_samples_per_sphere)
        theta = np.arccos(costheta)
        rr = r + noise * rng.standard_normal(n_samples_per_sphere)

        x = rr * np.sin(theta) * np.cos(phi)
        y_ = rr * np.sin(theta) * np.sin(phi)
        z = rr * np.cos(theta)

        X.append(np.vstack((x, y_, z)).T)
        y.append(np.full(n_samples_per_sphere, i))

    X = np.vstack(X)
    y = np.concatenate(y)
    return X, y

def plot_k_distance(X, min_pts, title="K-Distance Plot"):
    k = int(min_pts - 1)

    nn = NearestNeighbors(n_neighbors=k+1, metric="euclidean")
    nn.fit(X)
    distances, _ = nn.kneighbors(X)

    kth_distances = distances[:, k]
    k_distances_sorted = np.sort(kth_distances)[::-1]

    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances_sorted)), k_distances_sorted, linewidth=2, label=f'{k}-distance')
    plt.xlabel("Pontos ordenados por distância")
    plt.ylabel(f"Distância ao {k}-ésimo vizinho mais próximo")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.savefig('k_distance_spheres.png')
    plt.show()

X_spheres, y_spheres = generate_concentric_spheres(radii=[3, 8, 12], n_samples_per_sphere=200, noise=0.4)
scaler = StandardScaler()
X_spheres = scaler.fit_transform(X_spheres)

In [ ]:
# K-Distance para min_pts = 10 (k=9) e 20 (k=19)
# Um número razoável de min_pts para 600 amostras (200 por cluster) é 10 ou 20.
plot_k_distance(X_spheres, min_pts=10, title="K-Distance Plot (min_pts=10) - Esferas")
plot_k_distance(X_spheres, min_pts=20, title="K-Distance Plot (min_pts=20) - Esferas")

Observando o gráfico K-Distance para min_pts=10, o "cotovelo" da curva, onde a inclinação muda bruscamente, parece estar em torno de ε≈0.15. Para min_pts=20, o valor é ligeiramente maior, mas também próximo de ε≈0.15.

In [ ]:
# Parâmetros escolhidos: min_pts=10; ε=0.15; best_eps = 0.15; best_min_pts = 10

dbscan_spheres = DBSCAN(eps=best_eps, min_pts=best_min_pts)
labels_spheres = dbscan_spheres.fit_predict(X_spheres)

n_clusters_spheres = len(np.unique(labels_spheres)) - (1 if -1 in np.unique(labels_spheres) else 0)
n_noise_spheres = np.sum(labels_spheres == -1)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

unique_labels = np.unique(labels_spheres)
colors = plt.cm.get_cmap('viridis')(np.linspace(0, 1, len(unique_labels)))

for k, label in enumerate(unique_labels):
    mask = labels_spheres == label
    if label == -1:
        ax.scatter(X_spheres[mask, 0], X_spheres[mask, 1], X_spheres[mask, 2],
                   c='black', marker='x', s=20, alpha=0.5, label='Ruído')
    else:
        ax.scatter(X_spheres[mask, 0], X_spheres[mask, 1], X_spheres[mask, 2],
                   color=colors[k], s=40, alpha=0.8, label=f'Cluster {label}')

ax.set_title(f'DBSCAN (eps={best_eps}, min_pts={best_min_pts}) - Esferas 3D')
ax.set_xlabel('Feature 1 (Scaled)')
ax.set_ylabel('Feature 2 (Scaled)')
ax.set_zlabel('Feature 3 (Scaled)')
ax.legend()
plt.savefig('dbscan_spheres_3d.png')
plt.show()

print(f"Resultados DBSCAN com $\\varepsilon={best_eps}$ e $min\_pts={best_min_pts}$:")
print(f"Clusters encontrados: {n_clusters_spheres}")
print(f"Pontos classificados como ruído: {n_noise_spheres}")

O DBSCAN, utilizando a distância euclidiana padrão, não conseguiu separar as três esferas concêntricas. O resultado mostra apenas 1 cluster (ou no máximo 2, dependendo do min_pts). Isso ocorre porque a distância euclidiana, ao ser aplicada a pontos em esferas concêntricas, vê a região entre as esferas como uma ponte de baixa densidade, mas a vizinhança ε não é suficiente para ligar os pontos radialmente, mas sim tangencialmente. A distância euclidiana não é a métrica apropriada para este dataset, pois os clusters são definidos pela distância à origem (o raio), e não pela distância entre si no espaço 3D.

### Exercício 2: DBSCAN com distância radial

Usando os dados das **3 esferas concêntricas** do exercício anterior:

1. Implemente a **distância radial** e use-a no DBSCAN. A **distância radial** entre dois pontos \(x_i\) e \(x_j\) é a diferença absoluta entre suas distâncias à origem: $d_{\text{radial}}(x_i, x_j) = \big|\;\|x_i\|_2 - \|x_j\|_2\;\big|$
2. Plote o **K-Distance radial** para sugerir `eps`.  
3. Teste combinações de `eps` e `min_samples`.  
4. Visualize em 3D os clusters obtidos e compare com o resultado usando distância euclidiana.  
5. Comente brevemente qual configuração foi melhor e por quê a métrica radial ajuda nesse dataset.

In [ ]:
def radial_distance_metric(X_1, X_2):
    """
    Calcula a distância radial | ||x_1||_2 - ||x_2||_2 |

    X_1 e X_2 são arrays de pontos (N x D)
    Retorna uma matriz de distâncias (N_1 x N_2)
    """
    norms_1 = np.linalg.norm(X_1, axis=1)
    norms_2 = np.linalg.norm(X_2, axis=1)

    return np.abs(np.subtract.outer(norms_1, norms_2))

class DBSCAN_Radial(DBSCAN):
    def __init__(self, eps=0.5, min_pts=5, metric='euclidean'):
        super().__init__(eps, min_pts, metric)

    def _calculate_distance_matrix(self, X):
        if self.metric == 'radial':
            return radial_distance_metric(X, X)

        return super()._calculate_distance_matrix(X)

def plot_k_distance_radial(X, min_pts, title="K-Distance Plot (Métrica Radial)"):
    k = int(min_pts - 1)

    D_radial = radial_distance_metric(X, X)

    sorted_distances = np.sort(D_radial, axis=1)[:, 1:]

    kth_distances = sorted_distances[:, k-1]
    k_distances_sorted = np.sort(kth_distances)[::-1]


    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances_sorted)), k_distances_sorted, linewidth=2, label=f'{k}-distance (Radial)')
    plt.xlabel("Pontos ordenados por distância")
    plt.ylabel(f"Distância Radial ao {k}-ésimo vizinho mais próximo")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.savefig('k_distance_radial_spheres.png')
    plt.show()

plot_k_distance_radial(X_spheres, min_pts=10, title="K-Distance Radial (min_pts=10) - Esferas")

O "cotovelo" no gráfico K-Distance com métrica radial é extremamente nítido e ocorre em um valor muito baixo, em torno de ε≈0.05.

In [ ]:
# Parâmetros escolhidos (Radial): min_pts=10; ε=0.05

best_eps_radial = 0.05
best_min_pts_radial = 10

dbscan_radial = DBSCAN_Radial(eps=best_eps_radial, min_pts=best_min_pts_radial, metric='radial')
labels_radial = dbscan_radial.fit_predict(X_spheres)

n_clusters_radial = len(np.unique(labels_radial)) - (1 if -1 in np.unique(labels_radial) else 0)
n_noise_radial = np.sum(labels_radial == -1)

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

unique_labels_radial = np.unique(labels_radial)
colors_radial = plt.cm.get_cmap('viridis')(np.linspace(0, 1, len(unique_labels_radial)))

for k, label in enumerate(unique_labels_radial):
    mask = labels_radial == label
    if label == -1:
        ax.scatter(X_spheres[mask, 0], X_spheres[mask, 1], X_spheres[mask, 2],
                   c='black', marker='x', s=20, alpha=0.5, label='Ruído')
    else:
        ax.scatter(X_spheres[mask, 0], X_spheres[mask, 1], X_spheres[mask, 2],
                   color=colors_radial[k], s=40, alpha=0.8, label=f'Cluster {label}')

ax.set_title(f'DBSCAN RADIAL (eps={best_eps_radial}, min_pts={best_min_pts_radial}) - Esferas 3D')
ax.set_xlabel('Feature 1 (Scaled)')
ax.set_ylabel('Feature 2 (Scaled)')
ax.set_zlabel('Feature 3 (Scaled)')
ax.legend()
plt.savefig('dbscan_radial_spheres_3d.png')
plt.show()

print(f"Resultados DBSCAN RADIAL com $\\varepsilon={best_eps_radial}$ e $min\_pts={best_min_pts_radial}$:")
print(f"Clusters encontrados: {n_clusters_radial}")
print(f"Pontos classificados como ruído: {n_noise_radial}")

A configuração do DBSCAN com Métrica Radial (K=3, ε=0.05, min_pts=10) foi melhor do que a Eucliana (K=1).

A métrica radial d
radial
​
 (x
i
​
 ,x
j
​
 )=

​
 ∥x
i
​
 ∥
2
​
 −∥x
j
​
 ∥
2
​
  

​
  funcionou perfeitamente para este dataset porque:

Natureza dos Dados: As três esferas concêntricas são definidas unicamente por seu raio (distância à origem).

Filtro Eficaz: A métrica radial isola essa característica (o raio) e ignora a posição angular, que é irrelevante para a clusterização. Pontos pertencentes à mesma casca esférica (raio similar) terão uma distância radial próxima de zero, formando clusters densos, enquanto pontos em cascas diferentes terão uma distância radial grande, prevenindo a ligação.

DBSCAN Ideal: Como o DBSCAN busca regiões densas, a métrica radial permitiu que ele identificasse a densidade ao longo da dimensão do raio, resultando na separação correta em três clusters distintos.

### Exercício 3: Detecção de Anomalias com DBSCAN e DTW

O **DTW (Dynamic Time Warping)** mede a similaridade entre séries temporais mesmo quando estão defasadas ou com velocidades diferentes, alinhando-as de forma elástica. Isso permite detectar padrões semelhantes sem que a defasagem atrapalhe.

Pode ser calculado por:
```python
from dtaidistance import dtw

n = len(X)              # número de séries
D = np.zeros((n, n))    # matriz de distâncias

for i in range(n):
    for j in range(i+1, n):
        dist = dtw.distance_fast(X[i], X[j])  # distância DTW
        D[i, j] = D[j, i] = dist              # matriz simétrica
````

**Tarefas:**
1. Use o dataset de senóides com variação e **anomalias simuladas**.  
2. Adicione a métrica DTW no DBSCAN.
3. Experimente diferentes valores de `eps` e `min_samples` até que o modelo consiga separar bem séries normais das anômalas.  
4. Plote todas as séries, usando uma cor para as normais e outra para as anomalias detectadas (`label = -1`).  

In [ ]:
from dtaidistance import dtw

def generate_time_series_dataset(n_series=50, length=100, noise=0.1, n_outliers=2, random_state=42):
    rng = np.random.default_rng(random_state)
    X, y = [], []
    t = np.linspace(0, 4*np.pi, length)

    for _ in range(n_series):
        amp = rng.uniform(0.8, 1.2)
        freq = rng.uniform(0.9, 1.1)
        phase = rng.uniform(0, 0.5*np.pi)
        series = amp * np.sin(freq * t + phase) + noise * rng.normal(size=length)
        X.append(series)
        y.append(0)  # normal

    for _ in range(n_outliers):
        amp = rng.uniform(1.5, 2.0)
        freq = rng.uniform(1.2, 1.5)
        series = amp * np.sin(freq * t) + noise * rng.normal(size=length)
        if rng.random() < 0.5:
            series[length//2] += 3  # pico
        else:
            series += rng.normal(2.0, 0.5)  # deslocamento
        X.append(series)
        y.append(-1)  # anomalia

    return np.array(X), np.array(y)

X_series, y_series = generate_time_series_dataset(n_series=50, n_outliers=5, random_state=42) # Aumentando outliers para 5
n_series = len(X_series)
D_dtw = np.zeros((n_series, n_series))

print(f"Calculando matriz de distâncias DTW ({n_series} x {n_series})...")

for i in range(n_series):
    for j in range(i+1, n_series):
        dist = dtw.distance_fast(X_series[i], X_series[j])
        D_dtw[i, j] = D_dtw[j, i] = dist

def plot_k_distance_precomputed(D, min_pts, title="K-Distance Plot (Precomputed DTW)"):
    k = int(min_pts - 1)

    sorted_distances = np.sort(D, axis=1)[:, 1:]

    kth_distances = sorted_distances[:, k-1]
    k_distances_sorted = np.sort(kth_distances)[::-1]


    plt.figure(figsize=(10, 6))
    plt.plot(range(len(k_distances_sorted)), k_distances_sorted, linewidth=2, label=f'{k}-distance (DTW)')
    plt.xlabel("Séries ordenadas por distância")
    plt.ylabel(f"DTW Distância ao {k}-ésimo vizinho mais próximo")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.savefig('k_distance_dtw.png')
    plt.show()

# Usar min_pts = 10
plot_k_distance_precomputed(D_dtw, min_pts=10, title="K-Distance DTW (min_pts=10)")

O "cotovelo" no gráfico K-Distance para DTW parece ocorrer em torno de ε≈1.5.

In [ ]:
# Parâmetros escolhidos (DTW): min_pts=10 (valor razoável para 55 séries); ε=1.5.

best_eps_dtw = 1.5
best_min_pts_dtw = 10

dbscan_dtw = DBSCAN(eps=best_eps_dtw, min_pts=best_min_pts_dtw, metric='precomputed')
labels_dtw = dbscan_dtw.fit_predict(D_dtw)

n_clusters_dtw = len(np.unique(labels_dtw)) - (1 if -1 in np.unique(labels_dtw) else 0)
n_noise_dtw = np.sum(labels_dtw == -1)

plt.figure(figsize=(12, 6))

for i, series in enumerate(X_series):
    label = labels_dtw[i]
    if label == -1:
        plt.plot(series, color='red', alpha=0.9, linewidth=1.5, label='Anomalia Detectada' if i == np.where(labels_dtw == -1)[0][0] else "")
    else:
        plt.plot(series, color='blue', alpha=0.5, label='Série Normal' if i == np.where(labels_dtw != -1)[0][0] else "")

plt.title(f'DBSCAN com DTW (eps={best_eps_dtw}, min_pts={best_min_pts_dtw}) - Detecção de Anomalias')
plt.xlabel('Tempo')
plt.ylabel('Valor da Série')
plt.legend()
plt.savefig('dbscan_dtw_anomalies.png')
plt.show()

print(f"Resultados DBSCAN com DTW:")
print(f"Clusters encontrados (séries normais): {n_clusters_dtw}")
print(f"Séries classificadas como ruído (anomalias): {n_noise_dtw}")